In [ ]:
%load_ext autoreload
%autoreload 2
import requests
import json
import pandas as pd
from globalFW_API_interface import getGFWConfig, createGeostoreIDfromGeometry, getDataSetUrl, getDataSetFields, queryDeforestationAlerts, queryCO2NetFlux, getArea
from pathlib import Path
import datetime
import time

In [ ]:
getGFWConfig()

In [ ]:
gfwBaseURL = 'https://data-api.globalforestwatch.org/'
headers = {'content-type': 'application/json'}


In [ ]:
datasets = requests.get(url=f'{gfwBaseURL}datasets', headers=headers)


## draw patch on geojson.io, copy geometry part and add the "type" : "string", bit

In [ ]:
geostoreGeometry = '{ "geometry": { "type": "string", "coordinates": [ [ [   -64.48111132990033,              -7.26380551976348             ],            [              -64.5836061807908,              -7.807104232575085             ],             [              -63.56702459848742,              -7.813321216519867            ],            [              -63.48335533245408,              -7.245130631814632            ],            [              -64.48111132990033,              -7.26380551976348            ]          ]        ],        "type": "Polygon"      }}'


## create a second geostore ID for seeing some deforestation alerts

In [ ]:
createGeostoreIDfromGeometry()

In [ ]:
getArea()

## helper function to create data set URLs

In [ ]:
getDataSetUrl()

In [ ]:
getDataSetUrl(datasetName="gfw_integrated_alerts")

## helper function to get data set fields

In [ ]:
getDataSetFields(datasetURL=getDataSetUrl())

In [ ]:
getDataSetFields(datasetURL = getDataSetUrl(datasetName="gfw_integrated_alerts"))

## deforestation area does not work yet...

In [ ]:
applicationHeaders = {'content-type': 'application/json'}

def queryDeforestationAlertsArea(
        datasetURL: str = getDataSetUrl(datasetName="gfw_integrated_alerts"), 
        geoJSON: str = "86929d7e-512e-00fa-c4f1-f42776ec1c52",
        fromDate: str = "2021-04-01",
        endDate: str = "2023-04-30",
        field: str = 'umd_glad_sentinel2_alerts', #wur_radd_alerts
    ):
    config = getGFWConfig()
    headers = {'Authorization' : config['gfwAuthHeader']}  | applicationHeaders  
    data = {
        "sql" : f"SELECT '{field}_coverage__coverage' FROM results WHERE {field}__date >= '{fromDate}' AND {field}__date <='{endDate}'",
        "geostore_id" : geoJSON,   
        "geostore_origin" : "gfw"
    }
    requestedData=requests.get(url=f'{datasetURL}/query/json', params= data, headers=headers)
    return requestedData.json()

In [ ]:
queryDeforestationAlertsArea()

### deforestation alerts are working nicely

In [ ]:
queryDeforestationAlerts(datasetURL=getDataSetUrl(), geoJSON="f3ceac63-34cb-e9d6-e436-f0c42cafff10", fromDate="2023-04-01", confidence="high")

In [ ]:
queryDeforestationAlerts(datasetURL=getDataSetUrl(), geoJSON="f3ceac63-34cb-e9d6-e436-f0c42cafff10", fromDate="2023-05-01", endDate="2023-05-05", confidence="high")

## CO2 net flux query

In [ ]:
queryCO2NetFlux()

## get complete list of data sets with some metadata

In [ ]:
datasetList = []
for dataset in datasetsJSON['data']:
    thisDataSetDict = {}
    thisDataSetDict["ID"]=dataset['dataset']
    thisDataSetDict["NAME"] = dataset['metadata']['title']
    thisDataSetDict["RESOLUTION"] = dataset['metadata']['resolution']
    thisDataSetDict["REF"] = dataset['metadata'].get('learn_more',None)
    thisDataSetDict["FUNCTION"] = dataset['metadata'].get('function',None)
    thisDataSetDict["UPDATE"] = dataset['metadata'].get('update_frequency',None)
    datasetList.append(thisDataSetDict)


datsetDataFrame = pd.DataFrame(datasetList)   

In [ ]:
datsetDataFrame[0:245]

In [ ]:
datsetDataFrame.to_csv('gfw_datasets_overview.csv')

In [ ]:
datsetDataFrame.to_excel('gfw_datasets_overview.xlsx')

In [ ]:
datasetsJSON['data'][114]

In [ ]:
datasetsJSON['data'][137]

In [ ]:
datasetsJSON['data'][242]

In [ ]:
datasetsJSON['data'][149]

In [ ]:
datasetsJSON['data'][113]